# Organización del número de personas afiliadas a la seguridad social

## Limpieza de los datos

<b>Objetivo:</b> eliminar las líneas innecesarias, y organizar los encabezados de los documentos. De esta manera podremos unificar todos los datos posteriormente.

In [1]:
import os

def clean_data(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['actividad', 'Serie;;;', 'Sin Zona asignada']
    headers = "Serie;Codigo_municipio;Nombre;2022;2023\n"

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for file in csv_files:
        original_file = os.path.join(input_folder, file)
        clean_file = os.path.join(output_folder, file)
        with open(original_file, 'r') as f_input, open(clean_file, 'w') as f_output:
            f_output.write(headers) 
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = True
                    elif '-' in line:
                        line = line.replace('-', '0')
                    if not delete:
                        fields = line.strip().split(';')
                        f_output.write(';'.join(fields[:3] + fields[-3:]) + '\n')
                    # Eliminar filas posteriores al ser el último municipio
                    
                    if 'Zarzalejo' in line:
                        delete_next = True

input_folder = ".\original_data"
output_folder = ".\clean_data"

clean_data(input_folder, output_folder)


## Unión de los datos

<b>Objetivo:</b> unir todos los archivos de la carpeta en uno solo, con sus encabezados correspondientes. Guardar el <i>.csv</i> resultante en la carpeta <i>union_final</i> para poder añadir los datos al archivo final.

In [2]:
import os, csv

def join_data(folder_path, output_path):

    # Crear un diccionario para almacenar los datos unidos
    join_data = {}

    # Obtener la lista de archivos en la carpeta
    files = os.listdir(folder_path)

    for file in files:
        if file.endswith('.csv'):
            with open(os.path.join(folder_path, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')

                # Saltar la primera fila que contiene los headers
                next(csv_reader)

                # Obtener los nombres de los archivos que irán en los headers
                type = file.split('_')[0]
                subtype = file.split('.')[0].split('_')[-2:]
                    
                # Iterar sobre cada fila en el file CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""
                    
                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in join_data:
                        join_data[(serie, codigo_municipio, nombre)] = {}

                    # Obtener los datos de cada año para el intervalo de edad actual
                    for i, value in enumerate(row[3:5], start=2022):
                        # Establecer los nombres de los headers de los datos
                        if 'ss' in subtype:
                            key = f"{type}_{subtype[1]}_{i}"
                        else:
                            key = f"{type}_afiliados_{subtype[0]}_{subtype[1]}_{i}"

                        # Agregar los datos al diccionario enlazado
                        join_data[(serie, codigo_municipio, nombre)][key] = value

    # Escribir los datos unidos en un nuevo file CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')

        # Escribir los encabezados
        final_keys = []

        for key, values in join_data.items():
            if final_keys == []:
                final_keys.extend(values.keys())
        final_keys = [str(clave) for clave in final_keys]

        # Escribir los encabezados en el file CSV
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *final_keys])

        # Escribir los datos unidos
        for key, values in join_data.items():
            line = [key[0], key[1], key[2]]
            for clave in final_keys:
                line.append(values.get(clave, ))
            csv_writer.writerow(line)

folder_path = './clean_data'
output_path = '../union_final/data/afiliados_ss.csv'

join_data(folder_path, output_path)

